In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from Secrets import CLIENT_ID, CLIENT_SECRET, REDIRECT_URL
import pandas as pd
import chromadb
import numpy as np
import random       

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URL,
                                               scope="user-library-read"))

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="spotify_data")

In [3]:
playlist = sp.playlist_tracks('1Ob3QykC4dzf3rQwWxE9LV', market = 'CAN',fields='items' )
track_ids = []
for track in playlist['items']:
    if track['track'] is not None and track['track']['id'] is not None:
        track_ids.append(track['track']['id'])

for id in track_ids:
    all_audio_data = list(sp.audio_features(id)[0].values())
    audio_features = all_audio_data[:11] + all_audio_data[-1:]

In [ ]:
df = pd.read_csv('./dataset.csv')
# print(df.head())
df = df.drop(['artists', 'album_name', 'track_name', 'popularity', 'explicit', 'duration_ms', 'track_genre'], axis='columns')
df = df[df.columns[1:]]

# Convert DataFrame to a list of dictionaries
df_audio_data = df.values.tolist()
for i in range(len(df_audio_data)):
    print(i)
    collection.upsert(embeddings=df_audio_data[i][1:], ids=df_audio_data[i][0])


results = collection.query(query_embeddings=audio_features, n_results=4)
print(results)